In [2]:
import numpy as np
from scipy.stats import rv_discrete
#from scipy.stats import cdf
from scipy.sparse import csr_matrix
from scipy.optimize import linprog
from itertools import product

In [5]:
## Discretize_LOS

def discretize_los(los, T):
    L = None
    if isinstance(los, int):
        L = np.concatenate((np.ones(los, dtype=int), np.zeros(T - los, dtype=int)))
    elif isinstance(los, np.ndarray):
        if len(los) >= T:
            L = los
        else:
            L = np.concatenate((los, np.zeros(T - len(los), dtype=float)))
    elif isinstance(los, rv_discrete):
        cdf_vals = los.cdf(np.arange(T + 1))
        L = 1.0 - cdf_vals
    else:
        raise ValueError("Invalid length of stay distribution")
    
    # Replace NaN and inf values with 0
    L = np.nan_to_num(L)
    
    return L


In [6]:
## compute active patients at each node at each time
# active patients = number of initial patients at node i - discharged patients at time t + remaining patients arriving before t

def compute_active_null(initial_patients, discharged_patients, admitted_patients, L):
    N, T = admitted_patients.shape
    active_null = np.zeros((N, T))
    
    for i in range(N):
        for t in range(T):
            active_null[i, t] = (
                initial_patients[i]
                - np.sum(discharged_patients[i, :t+1])
                + np.sum(L[t-t_1] * admitted_patients[i, t_1] for t_1 in range(t+1))
            )
    
    return active_null

In [7]:
## check if the number of initial patients, discharged patients, and beds are within the total number of admitted patients by time T
def check_sizes(initial_patients, discharged_patients, admitted_patients, beds):
    N, T = admitted_patients.shape
    
    assert initial_patients.shape == (N,)
    assert discharged_patients.shape == (N, T)
    assert beds.shape[0] == N


Optional Constraints

In [9]:
## only send patients between connected locations

def enforce_adj(model, sent, adj_matrix):
    if adj_matrix is None:
        return
    
    N, _, T = sent.shape
    assert adj_matrix.shape == (N, N)

    for i in range(N):
        for j in range(N):
            if not adj_matrix[i, j]:
                for t in range(T):
                    model.addConstr(sent[i, j, t] == 0)

def enforce_no_artificial_overflow(model, no_artificial_overflow, active_patients, active_null, capacity):
    if no_artificial_overflow:
        N, T = active_null.shape
        for i in range(N):
            for t in range(T):
                if active_null[i, t] <= capacity[i, -1]:
                    model.addConstr(active_patients[i, t] <= capacity[i, -1])
    return

def enforce_no_worse_overflow(model, no_worse_overflow, active_patients, active_null, capacity):
    if no_worse_overflow:
        N, T = active_null.shape
        for i in range(N):
            for t in range(T):
                if active_null[i, t] >= capacity[i, -1]:
                    model.addConstr(active_patients[i, t] <= active_null[i, t])
    return



In [12]:
## enforce a minimum time between sending and receiving
def enforce_sendreceivegap(model, sent, sendreceive_gap):
    if sendreceive_gap > 0:
        N, _, T = sent.shape
        for t in range(T - 1):
            for i in range(N):
                model.add_sos([(sent[j, i, t], sent[i, j, min(t + sendreceive_gap, T - 1)]) for j in range(N)], 1)
                model.add_sos([(sent[i, j, min(t + sendreceive_gap, T - 1)], sent[j, i, t]) for j in range(N)], 1)
    return

In [13]:
## enforce an upper limit on the number of transfers per hospital-day

def enforce_transferbudget(model, sent, transfer_budget, total_transfer_budget):
    N, _, T = sent.shape
    for i in range(N):
        if not math.isinf(transfer_budget[i]):
            model.add_constraints(sum(sent[i, :, t] for t in range(T)) <= transfer_budget[i] for t in range(T))
    if not math.isinf(total_transfer_budget):
        model.add_constraint(sum(sent[i, j, t] for i in range(N) for j in range(N) for t in range(T)) <= total_transfer_budget)

Optional Penalties

In [15]:
# penalize total sent if enabled
def add_sent_penalty(model, sent, objective, sent_penalty):
    if sent_penalty > 0:
        N, _, T = sent.shape
        objective.expr += sent_penalty * sum(sent[i, j, t] for i in range(N) for j in range(N) for t in range(T))

# penalize costs between hospitals
def add_cost_penalty(model, sent, objective, cost_matrix):
    if cost_matrix is not None:
        objective.expr += sum(sum(sent[:, :, t] * cost_matrix) for t in range(sent.shape[2]))

# penalize non-smoothness in sent patients if enabled
def add_smoothness_penalty(model, sent, objective, smoothness_penalty):
    if smoothness_penalty > 0:
        N, _, T = sent.shape
        smoothness_dummy = {(i, j, t): model.var() for i in range(N) for j in range(N) for t in range(T-1)}
        model.add_component("smoothness_dummy", smoothness_dummy)
        model.add_constraints((sent[i, j, t] - sent[i, j, t+1] <= smoothness_dummy[i, j, t],
                               sent[i, j, t] - sent[i, j, t+1] >= -smoothness_dummy[i, j, t]) for i in range(N) for j in range(N) for t in range(T-1))
        objective.expr += smoothness_penalty * sum(smoothness_dummy.values())
        objective.expr += smoothness_penalty * sum(sent[:, :, 0])

# penalize non-smoothness in active patients if enabled
def add_active_smoothness_penalty(model, sent, objective, smoothness_penalty, active_patients):
    if smoothness_penalty > 0:
        N, _, T = sent.shape
        active_smoothness_dummy = {(i, t): model.var() for i in range(N) for t in range(T-1)}
        model.add_component("active_smoothness_dummy", active_smoothness_dummy)
        model.add_constraints((active_patients[i, t] - active_patients[i, t+1] <= active_smoothness_dummy[i, t],
                               active_patients[i, t] - active_patients[i, t+1] >= -active_smoothness_dummy[i, t]) for i in range(N) for t in range(T-1))
        objective.expr += smoothness_penalty * sum(active_smoothness_dummy.values())

# penalize non-smoothness in admitted patients if enabled
def add_admitted_smoothness_penalty(model, sent, objective, smoothness_penalty, admitted_patients):
    if smoothness_penalty > 0:
        N, _, T = sent.shape
        total_admitted = {(i, t): admitted_patients[i, t] - sum(sent[i, :, t]) + sum(sent[:, i, t]) for i in range(N) for t in range(T)}
        admitted_smoothness_dummy_l1 = {(i, t): model.var() for i in range(N) for t in range(T-1)}
        model.add_component("admitted_smoothness_dummy_l1", admitted_smoothness_dummy_l1)
        model.add_constraints((total_admitted[i, t] - total_admitted[i, t+1] <= admitted_smoothness_dummy_l1[i, t],
                               total_admitted[i, t] - total_admitted[i, t+1] >= -admitted_smoothness_dummy_l1[i, t]) for i in range(N) for t in range(T-1))
        admitted_smoothness_dummy_lmax = {(i): model.var() for i in range(N)}
        model.add_component("admitted_smoothness_dummy_lmax", admitted_smoothness_dummy_lmax)
        model.add_constraints((admitted_smoothness_dummy_l1[i, t] <= admitted_smoothness_dummy_lmax[i]) for i in range(N) for t in range(T-1))
        objective.expr += smoothness_penalty * sum(admitted_smoothness_dummy_l1.values())
        objective.expr += smoothness_penalty * T * sum(admitted_smoothness_dummy_lmax.values())

# add setup costs if enabled
def add_setup_cost(model, sent, objective, setup_cost):
    if setup_cost > 0:
        N, _, T = sent.shape
        setup_dummy = {(i, j): model.binary_var() for i in range(N) for j in range(i+1, N)}
        model.add_component("setup_dummy", setup_dummy)
        model.add_constraints((1 - setup_dummy[i, j], sum(sent[i, j, t] for t in range(T)) + sum(sent[j, i, t] for t in range(T)) in Set(initialization=(1.0, 1.0))) for i in range(N) for j in range(i+1, N))
        objective.expr += setup_cost * sum(setup_dummy.values())

# load balancing penalty
def add_loadbalancing_penalty(model, sent, objective, balancing_penalty, balancing_thresh, active_patients, capacity):
    if balancing_penalty > 0:
        N, _, T = sent.shape
        balancing_dummy = {(i, t): model.var() for i in range(N) for t in range(T)}
        model.add_component("balancing_dummy", balancing_dummy)
        model.add_constraints((balancing_dummy[i, t] >= (active_patients[i, t] / capacity[i, 0]) - balancing_thresh) for i in range(N) for t in range(T))
        objective.expr += balancing_penalty * sum(balancing_dummy.values())

# weight objective per-location by max load
def add_severity_weighting(model, sent, objective, severity_weighting, overflow, active_null, capacity):
    if severity_weighting and overflow is not None:
        N, _, T = sent.shape
        max_load_null = [max(active_null[i, :] / capacity[i, 0]) for i in range(N)]
        severity_weight = [9.0 if max_load_null[i] <= 1.0 else 0.0 for i in range(N)]
        objective.expr += sum(overflow[:, 0] * severity_weight)


In [17]:
import numpy as np
from pulp import *

def patient_redistribution(
        capacity,
        initial_patients,
        discharged_patients,
        admitted_patients,
        los,
        obj='minoverflow',
        adj_matrix=None,
        cost_matrix=None,
        capacity_cushion=0.0,
        no_artificial_overflow=False,
        no_worse_overflow=False,
        sent_penalty=0,
        smoothness_penalty=0,
        active_smoothness_penalty=0,
        admitted_smoothness_penalty=0,
        constrain_integer=False,
        capacity_weights=[],
        node_weights=[],
        objective_weights=[],
        transfer_budget=[],
        total_transfer_budget=float('inf'),
        sendreceive_gap=0,
        min_send_amt=0,
        balancing_thresh=1.0,
        balancing_penalty=0,
        severity_weighting=False,
        setup_cost=0,
        timelimit=float('inf'),
        mipgap=0.0,
        solver='default',
        threads=-1,
        verbose=False,
    ):

    if len(capacity.shape) == 1:
        capacity = np.expand_dims(capacity, axis=1)

    N, T = admitted_patients.shape
    C = capacity.shape[1]

    capacity = capacity * (1.0 - capacity_cushion)

    if not capacity_weights:
        capacity_weights = [1] * C
    if not node_weights:
        node_weights = [1] * N
    if not objective_weights:
        objective_weights = np.ones((N, C))

    if isinstance(objective_weights[0], int):
        objective_weights = [objective_weights] * N

    assert len(objective_weights) == N
    assert all(len(row) == C for row in objective_weights)

    objective_weights = np.array([[objective_weights[i][j] * node_weights[i] * capacity_weights[j] for j in range(C)] for i in range(N)])

    if not transfer_budget:
        transfer_budget = [float('inf')] * N

    assert obj in ['minoverflow', 'minbedoverflow', 'loadbalance']

    L = discretize_los(los, T)

    model = LpProblem("Patient_Redistribution", LpMinimize)

    if constrain_integer and (timelimit > 0) and not np.isinf(timelimit):
        model.solver.timeout = timelimit

    if constrain_integer and (mipgap > 0):
        model.solver.mip.gap_limit = mipgap

    if solver != 'default':
        solver_lookup = {'auto': -1, 'primalsimplex': 0, 'dualsimplex': 1, 'barrier': 2, 'all': 3}
        model.solver.method = solver_lookup.get(solver, -1)

    if threads > 0:
        model.solver.threads = threads

    sent = LpVariable.dicts("sent", ((i, j, t) for i in range(1, N+1) for j in range(1, N+1) for t in range(1, T+1)), lowBound=0, cat="Continuous" if not constrain_integer else "Integer")

    def _active_patients_rule(i, t):
        return (
            initial_patients[i-1]
            - sum(discharged_patients[i-1, k-1] for k in range(1, t+1))
            + sum(L[t-j+1] * (
                admitted_patients[i-1, j-1]
                - sum(sent[(i, k, j)] for k in range(1, N+1))
                + sum(sent[(k, i, j)] for k in range(1, N+1))
            ) for j in range(1, t+1))
        )

    active_patients = {(i, t): _active_patients_rule(i, t) for i in range(1, N+1) for t in range(1, T+1)}
    active_null = compute_active_null(initial_patients, discharged_patients, admitted_patients, L)

    objective = lpSum([0])

    if obj == 'minoverflow':

        overflow = {(i, t, c): LpVariable("overflow_{}_{}_{}".format(i, t, c), lowBound=0, cat="Continuous") for i in range(1, N+1) for t in range(1, T+1) for c in range(1, C+1)}
        for i in range(1, N+1):
            for t in range(1, T+1):
                for c in range(1, C+1):
                    model += overflow[(i, t, c)] >= active_patients[(i, t)] - capacity[i-1, c-1]

        objective += lpSum([overflow[(i, t, c)] for i in range(1, N+1) for t in range(1, T+1) for c in range(1, C+1)]) * objective_weights

    elif obj == 'minbedoverflow':

        overflow = {(i, c): LpVariable("overflow_{}_{}".format(i, c), lowBound=0, cat="Continuous") for i in range(1, N+1) for c in range(1, C+1)}
        for i in range(1, N+1):
            for t in range(1, T+1):
                for c in range(1, C+1):
                    model += overflow[(i, c)] >= active_patients[(i, t)] - capacity[i-1, c-1] 

        objective += lpSum([overflow[(i, c)] for i in range(1, N+1) for c in range(1, C+1)]) * objective_weights

    elif obj == 'loadbalance':

        load_objective = {(i, t, c): LpVariable("load_objective_{}_{}_{}".format(i, t, c), lowBound=0, cat="Continuous") for i in range(1, N+1) for t in range(1, T+1) for c in range(1, C+1)}

        load = {(i, t, c): ((1.0 if capacity[i-1, c-1] == 0 else active_patients[(i, t)] / capacity[i-1, c-1])) for i in range(1, N+1) for t in range(1, T+1) for c in range(1, C+1)}
        for i in range(1, N+1):
            for t in range(1, T+1):
                for c in range(1, C+1):
                    model += load[(i, t, c)] - (lpSum([load[(k, t, c)] for k in range(1, N+1)])/N) <= load_objective[(i, t, c)]
                    model += -load[(i, t, c)] + (lpSum([load[(k, t, c)] for k in range(1, N+1)])/N) <= load_objective[(i, t, c)]

        objective += lpSum([load_objective[(i, t, c)] for i in range(1, N+1) for t in range(1, T+1) for c in range(1, C+1)]) * capacity_weights

    else:
        raise ValueError("Invalid objective")

    for i in range(1, N+1):
        for t in range(1, T+1):
            model += active_patients[(i, t)] >= 0

    for t in range(1, T+1):
        for i in range(1, N+1):
            model += lpSum([sent[(i, j, t)] for j in range(1, N+1)]) <= admitted_patients[i-1, t-1]

    # Optional Constraints/Costs
    # Implement your optional constraints here
    # Enforce adjacency
    if adj_matrix is not None:
        enforce_adj(model, sent, adj_matrix)

    # Enforce no artificial overflow
    if no_artificial_overflow:
        enforce_no_artificial_overflow(model, active_patients, active_null, capacity)

    # Enforce no worse overflow
    if no_worse_overflow:
        enforce_no_worse_overflow(model, active_patients, active_null, capacity)

    # Enforce send-receive gap
    if sendreceive_gap > 0:
        enforce_sendreceivegap(model, sent, sendreceive_gap)

    # Enforce transfer budget
    if transfer_budget is not None:
        enforce_transferbudget(model, sent, transfer_budget, total_transfer_budget)

    # Add sent penalty
    if sent_penalty > 0:
        add_sent_penalty(model, sent, objective, sent_penalty)

    # Add cost penalty
    if cost_matrix is not None:
        add_cost_penalty(model, sent, objective, cost_matrix)

    # Add smoothness penalty
    if smoothness_penalty > 0:
        add_smoothness_penalty(model, sent, objective, smoothness_penalty)

    # Add active smoothness penalty
    if active_smoothness_penalty > 0:
        add_active_smoothness_penalty(model, sent, objective, active_smoothness_penalty, active_patients)

    # Add admitted smoothness penalty
    if admitted_smoothness_penalty > 0:
        add_admitted_smoothness_penalty(model, sent, objective, admitted_smoothness_penalty, admitted_patients)

    # Add setup cost
    if setup_cost > 0:
        add_setup_cost(model, sent, objective, setup_cost)

    # Add load balancing penalty
    if balancing_penalty > 0:
        add_loadbalancing_penalty(model, sent, objective, balancing_penalty, balancing_thresh, active_patients, capacity)

    # Add severity weighting
    if severity_weighting:
        add_severity_weighting(model, sent, objective, severity_weighting, overflow, active_null, capacity) 


    # Solve
    model.solve()

    return model
